In [ ]:
from utils import *
from variables import *
from datetime import datetime, timedelta
import requests
import re
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pyhtml2pdf import  converter
import shutil
from plotly.colors import n_colors
from typing import Union, List, Dict
from bs4 import BeautifulSoup
import traceback
from utils import read_pickle, save_as_pickle
from generate_files import generate_pdfs, generate_parquets
from openpyxl.reader.excel import load_workbook
from openpyxl.utils.cell import coordinate_to_tuple

In [ ]:
# openpyxl will complain because extension is not .xlsx

auswahl = 1
excel_path = "./excel_files/STOXX_OI-Abfrage 2023-09-06 V9.00.xlsm"

#auswahl = 0
#excel_path = "./excel_files/EUREX 2023-08-28.xlsm"


wb = load_workbook(excel_path, data_only=True, keep_vba=True)
Ueberhaenge_sheet = wb['Ueberhaenge']
heute = Ueberhaenge_sheet.cell(*coordinate_to_tuple("G5")).value
expiry = Ueberhaenge_sheet.cell(*coordinate_to_tuple("G3")).value
expiry_1 = Ueberhaenge_sheet.cell(*coordinate_to_tuple("R6")).value
stock_price = Ueberhaenge_sheet.cell(*coordinate_to_tuple("X4")).value
InterestRate = Ueberhaenge_sheet.cell(*coordinate_to_tuple("N3")).value
ZentralKurs =  Ueberhaenge_sheet.cell(*coordinate_to_tuple("C3")).value   #round(stock_price/span)*span

tage_bis_verfall = (expiry - heute).days #+ 1

print(f"tage_bis_verfall = {tage_bis_verfall}")
print(f"stock_price = {stock_price}")
print(f"InterestRate = {InterestRate}")



dict_auswahl_prefix = {
    0 : "",
    1 : "STOXX_"
}


dict_prod_bus = {}
for productdate_idx in [0,1]:
    dict_prod_bus[productdate_idx] = {}
    for busdate_idx in [0,1]:
        print(f"busdate_idx = {busdate_idx}   | productdate_idx = {productdate_idx}")
        if (busdate_idx == 1) and (productdate_idx == 1):
            # No need to request data for this case so we skip this iteration
            break

        if (busdate_idx == 0) and (productdate_idx == 0):
            contractsCall_aux_df =  pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Call_Front")
            contractsPut_aux_df =  pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Put_Front")

        if (busdate_idx == 1) and (productdate_idx == 0):
            contractsCall_aux_df= pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}CallFront-1")
            contractsPut_aux_df = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}PutFront-1")

        if (busdate_idx == 0) and (productdate_idx == 1):
            contractsCall_aux_df = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Put+01")
            contractsPut_aux_df= pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Call+01")


        aux_df = contractsCall_aux_df[['strike','openInterest']].merge(
            contractsPut_aux_df[['strike','openInterest']],
            on = "strike",
            how="left",
            suffixes=('_CF', '_PF')
        )
        dict_prod_bus[productdate_idx][busdate_idx] = aux_df

In [ ]:
print(f"heute = {heute}")
# Create results folder in case they still were not created
list_folder_results =  [current_results_path, old_results_path, temp_results_path]
for folder in list_folder_results:
    create_folder(folder)


if auswahl == 0: # DAX
    Spannweite = 2000
    Schritt = 50
    volatility_Laufzeit = 60
    KontraktWert = 5

else: # STOXX
    Spannweite = 700
    Schritt = 25
    volatility_Laufzeit = 365
    KontraktWert = 1

span = (Spannweite/4)

Minkurs = ZentralKurs - (Spannweite/ 2)

Maxkurs = Minkurs + Spannweite
Schritte = int(Spannweite / Schritt)

DetailMin = round(stock_price - (Spannweite)/4)
DetailMax = DetailMin + (Spannweite/2)


if tage_bis_verfall >= 1:
    delta = 0.5
else:
    delta = 1

print(f"stock_price = {stock_price}")
print(f"Minkurs = {Minkurs}")
print(f"Maxkurs = {Maxkurs}")
print(f"Schritte = {Schritte}")
print(f"ZentralKurs = {ZentralKurs}")
print(f"volatility_Laufzeit = {volatility_Laufzeit}")


# Second stage
# Create Basis column that will be used for multiple tables
Basis = pd.Series((Minkurs + np.arange(int(Schritte) +1)* Schritt)[::-1])

# Transform series into dataframe to make dataframe creation easier 
Basis_df = Basis.reset_index().rename(columns={0:"Basis"})[["Basis"]]


# Ueberhaenge_df, Summery_df are created from a copy of the same Dataframe
# so they have the same index. It means they could be in one  Dataframe but
#  I decided to keep them separated as they were in the VBA code.

Ueberhaenge_df = Basis_df.copy()
Summery_df = Basis_df.copy()

for productdate_idx in [0,1]:
    for busdate_idx in [0,1]:

        if (busdate_idx == 1) and (productdate_idx == 1):
            continue

        # Create dataframe with the just requested data
        aux_df = Basis_df.copy()

        aux_df = aux_df.merge(
            dict_prod_bus[productdate_idx][busdate_idx],
            left_on = "Basis",
            right_on = "strike",
            how= "left"
        )

        if productdate_idx == 0:
            Ueberhaenge_df[busdate_idx] = aux_df.openInterest_PF - aux_df.openInterest_CF

            if busdate_idx == 0:
                Summery_df["openInterest_PF"] = aux_df["openInterest_PF"]
                Summery_df["openInterest_CF"] = aux_df["openInterest_CF"]
        
        elif productdate_idx == 1:
            if busdate_idx == 0:
                Ueberhaenge_df["nextContract"] = aux_df.openInterest_PF - aux_df.openInterest_CF
                Summery_df["nextContract"] = Ueberhaenge_df["nextContract"]


Ueberhaenge_df.rename(columns = {0 : "Front"},inplace=True)
Ueberhaenge_df.rename(columns = {1 : "Last"},inplace=True)


Summery_df["heute"] = Ueberhaenge_df["Front"] * (1 / KontraktWert) * delta
Summery_df["last_day"] = Ueberhaenge_df["Last"] * (1 / KontraktWert) * delta

Ueberhaenge_df["Summe"] = Ueberhaenge_df[["Front", "nextContract"]].sum(axis=1)
Ueberhaenge_df = Ueberhaenge_df[['Basis', 'Summe',"Last",'Front', "nextContract"]]


Summery_df["Änderung"] = Summery_df.heute - Summery_df.last_day

if (delta == 1):
    Summery_df['nextContract'] =  Summery_df['nextContract'] / 2

SummeryDetail_df = Summery_df[(Summery_df.Basis >= DetailMin) & (Summery_df.Basis < (DetailMax + Schritt))]

# Third stage
SchrittWeite = 10
InterestRate = 3.798/100
volatility = 16.35/100

print(f"volatility = {volatility}")
print(f"InterestRate = {InterestRate}")

In [ ]:
Tage = tage_bis_verfall
if Tage == 0:
    Tage = 0.5

Tage_1 = (expiry_1 - heute).days +1
Kurs_count = int(Spannweite/SchrittWeite)+1 
HedgeBedarf_kurs=  pd.DataFrame(Maxkurs - np.arange(Kurs_count)* SchrittWeite,columns= ["Basis"])


Hedge_dimensions = Kurs_count,int(Schritte+1)
HedgeBedarf_values = np.zeros(Hedge_dimensions)
HedgeBedarf1_values = np.zeros(Hedge_dimensions)


for k in range(Hedge_dimensions[1]):
    Basis_value = Basis[k]
    Kontrakte = Ueberhaenge_df.loc[k,"Front"]
    Kontrakte_1 = Ueberhaenge_df.loc[k,"nextContract"]
    for i in range(Hedge_dimensions[0]):
        Kurs = Maxkurs - i * SchrittWeite
        #print(Kurs)
        # In python np.log = natural log
        h1 = np.log(Kurs / Basis_value)
        if auswahl == 0:
            sigma = volatility * ((Tage / volatility_Laufzeit) ** 0.5)
        else:
            sigma = volatility

        sigma_1 = volatility * ((Tage_1 / volatility_Laufzeit) ** 0.5)
        h2 = InterestRate + sigma * sigma / 2
        h2_1 = InterestRate + sigma_1 * sigma_1 / 2
        d1 = (h1 + (h2 * (Tage / 365))) / (sigma * ((Tage / 365) ** 0.5))
        d1_1 = (h1 + (h2_1 * (Tage_1 / 365))) / (sigma_1 * ((Tage_1 / 365) ** 0.5))
        Phi = norm.pdf(d1, 0, 1)               
        Phi_1 = norm.pdf(d1_1 + 0.01, 0, 1)
        Gamma = Phi / (Kurs * (sigma * (Tage / 365) ** 0.5))
        Gamma_1 = Phi_1 / (Kurs * (sigma_1 * (Tage_1 / 365) ** 0.5))
        
        HedgeBedarf_values[i,k] = Gamma * Kontrakte / KontraktWert
        HedgeBedarf1_values[i,k] = Gamma_1 * Kontrakte_1 / KontraktWert


HedgeSum = HedgeBedarf_values.sum(axis=1)/2
HedgeSum_1 = HedgeBedarf1_values.sum(axis=1)/2

HedgeBedarf_df = pd.DataFrame(data =HedgeBedarf_values, columns= Basis )
HedgeSum_df = pd.DataFrame(HedgeSum,columns=["HedgeSum"])
HedgeBedarf_df = pd.concat([HedgeBedarf_kurs,HedgeSum_df,HedgeBedarf_df], axis=1)

HedgeBedarf1_df = pd.DataFrame(data =HedgeBedarf1_values, columns= Basis)
HedgeSum1_df = pd.DataFrame(HedgeSum_1,columns=["HedgeSum"])
HedgeBedarf1_df = pd.concat([HedgeBedarf_kurs,HedgeSum1_df,HedgeBedarf1_df], axis=1)

# Fourth Stage

In [ ]:
HedgeBedarf_excel = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}HedgeBedarf",skiprows=3).rename(columns = {"Unnamed: 0" : "Basis", "Unnamed: 2" : "HedgeSum"})[['Basis', 'HedgeSum']]
HedgeBedarf_excel.head()

In [ ]:
compare_df = HedgeBedarf_df[['Basis', 'HedgeSum']].merge(HedgeBedarf_excel, on = 'Basis',suffixes=('_py', '_excel'))

In [ ]:
# Biggest difference between python and excel
(compare_df.HedgeSum_py - compare_df.HedgeSum_excel).abs().max()